# Creating Data

In [1]:
import requests

#API_KEY = 0CUKFG69H3TWK38Y
#STOCK SYMBOL = NVDA
# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=NVDA&outputsize=full&apikey=0CUKFG69H3TWK38Y'   # replace API KEY and stock symbol
r = requests.get(url)
data = r.json()

In [2]:
import json
import csv

# Sample JSON data as a string (you would replace this with loading the JSON from a file)
json_data = '''{
    "Meta Data": {
        "1. Information": "Daily Prices (open, high, low, close) and Volumes",
        "2. Symbol": "IBM",
        "3. Last Refreshed": "2024-10-24",
        "4. Output Size": "Compact",
        "5. Time Zone": "US/Eastern"
    },
    "Time Series (Daily)": {
        "2024-10-24": {"1. open": "220.8000", "2. high": "221.3200", "3. low": "216.1600", "4. close": "218.3900", "5. volume": "11193440"},
        "2024-10-23": {"1. open": "223.1000", "2. high": "224.5000", "3. low": "217.0000", "4. close": "220.0000", "5. volume": "12800000"}
        // more dates can be added here
    }
}'''

# Load JSON data
#data = json.loads(json_data)

# Extract "Time Series (Daily)" part
time_series = data.get("Time Series (Daily)", {})

# Prepare data for CSV
csv_data = [("Date", "Close")]  # Header for CSV file

# Loop through each date and get the "close" price
for date, daily_data in time_series.items():
    close_price = daily_data.get("4. close", None)
    if close_price:
        csv_data.append((date, close_price))

# Write data to CSV file
csv_file = 'Stock_prices.csv'
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerows(csv_data)

print(f"Data saved to {csv_file}")

Data saved to Stock_prices.csv


# Preprocessing

In [3]:
import pandas as pd
import numpy as np
from scipy.ndimage import gaussian_filter1d
from scipy.signal import savgol_filter

# Ανάγνωση των δεδομένων 
data = pd.read_csv('Stock_prices.csv')
print(data.head())

# Έλεγχος για το αν υπάρχουν NULL τιμές 
print(data.isna().sum())
# Έλεγχος τύπων δεδομένων
print(data.dtypes)

         Date    Close
0  2024-10-30  139.335
1  2024-10-29  141.250
2  2024-10-28  140.520
3  2024-10-25  141.540
4  2024-10-24  140.410
Date     0
Close    0
dtype: int64
Date      object
Close    float64
dtype: object


In [4]:
# Μετατροπή του index column 
data['Date']=pd.to_datetime(data['Date'])
data.set_index('Date',inplace=True)

In [5]:
# Εφαρμογή Gaussian Φίλτρου

# Καθορισμός σ (τυπική απόκλιση) για το Gaussian φίλτρο
sigma = 2
# Εφαρμογή Gaussian φίλτρου στην στήλη Close
data['Close'] = gaussian_filter1d(data['Close'].values, sigma=sigma)

In [6]:
data.head()

,Close
Date,
2024-10-30,140.380543
2024-10-29,140.492404
2024-10-28,140.657693
2024-10-25,140.826575
2024-10-24,140.979034


In [7]:
# Εφαρμογή Savitzky-Golay Φίλτρου

# Παράδειγμα παραμέτρων
window_length = 5  # Το μήκος του παραθύρου
polyorder = 2      # Τάξη του πολυωνύμου

# Εφαρμογή του φίλτρου Savitzky-Golay στην στήλη Close
data['Close'] = savgol_filter(data['Close'].values, window_length=window_length, polyorder=polyorder)

In [8]:
data.head()

,Close
Date,
2024-10-30,140.373132
2024-10-29,140.508078
2024-10-28,140.655137
2024-10-25,140.831076
2024-10-24,140.989065


# Lagged Data

In [11]:
New_data = data.copy()
New_data_col = []

# Δημιουργία καθυστερημένων στηλών
lagged_col_data = {}  # Λεξικό για αποθήκευση δεδομένων καθυστερημένων στηλών
for i in range(1, 71):
    for col in ['Close']:
        lagged_col_name = f"{col}_{i}"  # Δημιουργία ονόματος καθυστερημένης στήλης
        lagged_col_data[lagged_col_name] = New_data[col].shift(-i)  # Δημιουργία καθυστερημένων στηλών
        New_data_col.append(lagged_col_name)

# Συγχώνευση δεδομένων καθυστερημένων στηλών
New_data = pd.concat([New_data, pd.DataFrame(lagged_col_data)], axis=1)

# Συμπλήρωση τιμών NaN με 0
New_data.fillna(0, inplace=True)

# Αφαίρεση της στήλης 'Close' από το DataFrame
New_data_column = New_data.pop('Close')

# Εισαγωγή της στήλης 'label' στο τέλος του νέου Data Frame 
New_data['Close'] = New_data_column

# Εμφάνιση του πρώτου κομματιού του καθυστερημένου DataFrame για έλεγχο των αλλαγών
print(New_data.head())
print(New_data.tail())

               Close_1     Close_2     Close_3     Close_4     Close_5  \
Date                                                                     
2024-10-30  140.508078  140.655137  140.831076  140.989065  141.043337   
2024-10-29  140.655137  140.831076  140.989065  141.043337  140.810049   
2024-10-28  140.831076  140.989065  141.043337  140.810049  140.112037   
2024-10-25  140.989065  141.043337  140.810049  140.112037  138.963383   
2024-10-24  141.043337  140.810049  140.112037  138.963383  137.626311   

               Close_6     Close_7     Close_8     Close_9    Close_10  ...  \
Date                                                                    ...   
2024-10-30  140.810049  140.112037  138.963383  137.626311  136.448231  ...   
2024-10-29  140.112037  138.963383  137.626311  136.448231  135.617052  ...   
2024-10-28  138.963383  137.626311  136.448231  135.617052  135.044773  ...   
2024-10-25  137.626311  136.448231  135.617052  135.044773  134.452492  ...   
2024-10

# Train & Validation Data 

In [12]:
from sklearn.model_selection import train_test_split
# Δεδομένα εκπαίδευσης
Train_data = New_data[New_data.index.year <= 2023]
print(Train_data)
# Δεδομένα επικύρωσης 
Valid_test_data = New_data[New_data.index.year > 2023]
print(Valid_test_data)

               Close_1     Close_2     Close_3     Close_4     Close_5  \
Date                                                                     
2023-12-29  490.667163  491.441291  491.244116  490.624552  490.261232   
2023-12-28  491.441291  491.244116  490.624552  490.261232  490.384625   
2023-12-27  491.244116  490.624552  490.261232  490.384625  490.534423   
2023-12-26  490.624552  490.261232  490.384625  490.534423  489.869318   
2023-12-22  490.261232  490.384625  490.534423  489.869318  487.819726   
...                ...         ...         ...         ...         ...   
1999-11-05   27.574503   26.767430   26.065782   25.398109    0.000000   
1999-11-04   26.767430   26.065782   25.398109    0.000000    0.000000   
1999-11-03   26.065782   25.398109    0.000000    0.000000    0.000000   
1999-11-02   25.398109    0.000000    0.000000    0.000000    0.000000   
1999-11-01    0.000000    0.000000    0.000000    0.000000    0.000000   

               Close_6     Close_7   

In [13]:
X_train = Train_data.drop(['Close'], axis=1)
y_train = Train_data['Close']

In [14]:
X = Valid_test_data.drop(['Close'], axis=1)  # Εξαίρεση της τελευταίας στήλης, που είναι η ετικέτα κλεισίματος της μςτοχής 
y = Valid_test_data['Close']

X_val, X_test, y_val, y_test = train_test_split(X, y, test_size=0.4)

# Linear Model

In [15]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Δημιουργία και εκπαίδευση του γραμμικού μοντέλου
model = LinearRegression()
model.fit(X_train, y_train)

coefficients = model.coef_
intercept = model.intercept_
print("Συντελεστές:", coefficients)
print("Βασική τιμή:", intercept)

Συντελεστές: [ 2.50535595e+00 -1.78102930e+00 -2.02558953e-01  4.25621395e-01
  2.60061733e-01 -8.86519056e-02 -1.72324208e-01 -3.51421412e-02
  7.27606866e-02  5.24121355e-02 -1.18165293e-02 -3.11119847e-02
 -9.73331486e-03  7.98803687e-03  8.04251840e-03  4.28080721e-03
  3.36569411e-03 -6.70604176e-04 -8.16637875e-03 -9.69568695e-03
 -5.97539139e-04  1.02466568e-02  1.02352279e-02 -9.67310517e-04
 -9.84541740e-03 -5.84082520e-03  4.65760695e-03  7.33017265e-03
 -1.89167292e-04 -6.56312051e-03 -4.26415105e-03  1.59861705e-03
  4.18558158e-03  4.15795620e-03  3.14993508e-03 -1.78142815e-03
 -9.43008377e-03 -9.32110366e-03  3.84473082e-03  1.66135367e-02
  1.07312200e-02 -1.00578578e-02 -2.00349549e-02 -4.08440571e-03
  1.92631816e-02  1.91333832e-02 -6.26875195e-03 -2.45482364e-02
 -1.12297662e-02  1.66659303e-02  2.27861218e-02 -2.34212489e-04
 -2.21100930e-02 -1.57635211e-02  8.78525987e-03  2.00565613e-02
  5.70611342e-03 -1.44046316e-02 -1.52287269e-02  3.16613240e-03
  1.80226542

In [16]:
# Πρόβλεψη επάνω στα δεδομένα επικύρωσης 
y_val_pred = model.predict(X_val)

# Υπολογισμός μετρικών αξιολόγησης
mse_val = mean_squared_error(y_val, y_val_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
r2_val = r2_score(y_val, y_val_pred)

# Εκτύπωση αποτελεσμάτων
print(f'Validation MSE: {mse_val}')
print(f'Validation MAE: {mae_val}')
print(f'Validation R²: {r2_val}')

Validation MSE: 12.714330476780843
Validation MAE: 1.6295077129890745
Validation R²: 0.999907223598682


In [17]:
# Πρόβλεψη επάνω στα δεδομένα ελέγχου 
y_test_pred = model.predict(X_test)

# Υπολογισμός μετρικών αξιολόγησης
mse_test = mean_squared_error(y_test, y_test_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)
r2_test = r2_score(y_test, y_test_pred)

# Εκτύπωση αποτελεσμάτων
print(f'Test MSE: {mse_test}')
print(f'Test MAE: {mae_test}')
print(f'Test R²: {r2_test}')

Test MSE: 35.66146394508081
Test MAE: 2.231383687181066
Test R²: 0.9996980244151945
